#### 导入必要的库

In [15]:
import cv2
import numpy as np
from bitstream import BitStream
import huffmanEncode

#### Z字形扫描、亮度量化表，色度量化表(未使用)

In [16]:
zigzagOrder = np.array(
[  0,  1,  8, 16,  9,  2,  3, 10,
  17, 24, 32, 25, 18, 11,  4,  5,
  12, 19, 26, 33, 40, 48, 41, 34,
  27, 20, 13,  6,  7, 14, 21, 28,
  35, 42, 49, 56, 57, 50, 43, 36,
  29, 22, 15, 23, 30, 37, 44, 51,
  58, 59, 52, 45, 38, 31, 39, 46,
  53, 60, 61, 54, 47, 55, 62, 63])

std_luminance_quant_tbl = np.array(
[ 16,  11,  10,  16,  24,  40,  51,  61,
  12,  12,  14,  19,  26,  58,  60,  55,
  14,  13,  16,  24,  40,  57,  69,  56,
  14,  17,  22,  29,  51,  87,  80,  62,
  18,  22,  37,  56,  68, 109, 103,  77,
  24,  35,  55,  64,  81, 104, 113,  92,
  49,  64,  78,  87, 103, 121, 120, 101,
  72,  92,  95,  98, 112, 100, 103,  99],dtype=int)

# 灰度图像编码。色度量化表未使用
'''
std_chrominance_quant_tbl = np.array(
[ 17,  18,  24,  47,  99,  99,  99,  99,
  18,  21,  26,  66,  99,  99,  99,  99,
  24,  26,  56,  99,  99,  99,  99,  99,
  47,  66,  99,  99,  99,  99,  99,  99,
  99,  99,  99,  99,  99,  99,  99,  99,
  99,  99,  99,  99,  99,  99,  99,  99,
  99,  99,  99,  99,  99,  99,  99,  99,
  99,  99,  99,  99,  99,  99,  99,  99],dtype=int)
'''

'\nstd_chrominance_quant_tbl = np.array(\n[ 17,  18,  24,  47,  99,  99,  99,  99,\n  18,  21,  26,  66,  99,  99,  99,  99,\n  24,  26,  56,  99,  99,  99,  99,  99,\n  47,  66,  99,  99,  99,  99,  99,  99,\n  99,  99,  99,  99,  99,  99,  99,  99,\n  99,  99,  99,  99,  99,  99,  99,  99,\n  99,  99,  99,  99,  99,  99,  99,  99,\n  99,  99,  99,  99,  99,  99,  99,  99],dtype=int)\n'

#### 导入图像名称，导出图像名称，质量因子

In [17]:
inputImgname="lena_gray.bmp"
outputImgname="lena_85.jpeg"
quality=85

#### 根据输入的质量因子缩放量化表

In [18]:
quality=np.clip(quality,1,100)  #将输入的质量因子规约到[1,100]
if quality>=50:
    quality=200-2*quality
else:
    quality=5000/quality
luminance_quant_tbl=np.array(np.floor((quality*std_luminance_quant_tbl+50)/100))
luminance_quant_tbl=np.where(luminance_quant_tbl<1,1,luminance_quant_tbl)
luminance_quant_tbl=np.where(luminance_quant_tbl>255,255,luminance_quant_tbl)
luminance_quant_tbl=luminance_quant_tbl.reshape([8,8]).astype(np.int32)
# print(luminance_quant_tbl)

In [19]:
srcImage=cv2.imread(inputImgname,0) # 0：读取的图像为灰度图
srcWidth,srcHeight=srcImage.shape[:2]

#### 将原始图像的宽高扩展为8的整数倍，便于后续编码

In [20]:
imgWidth,imgHeight=srcWidth,srcHeight
if srcWidth%8!=0:
    imgWeight=srcWidth//8*8+8
if srcHeight%8!=0:
    imgHeight=srcHeight//8*8+8
padImage=np.zeros(shape=(imgWidth,imgHeight),dtype=np.uint8)
padImage[:srcWidth,:srcHeight]=srcImage


#### 点位平移

In [21]:
padImage=padImage.astype(np.int32)-127

In [ ]:
before=0    #记录DC系数差分编码的前一个DC系数
DCs=0       #记录当前DC系数值
flag=0
sosBitStream = BitStream()
for i in range(imgWidth//8):
    for j in range(imgHeight//8):
        # 获取一个编码块
        tmp=padImage[i*8:(i+1)*8,j*8:(j+1)*8].astype(np.float64)

        # DCT变换
        tmp=cv2.dct(tmp)

        # 量化表量化后四舍五入
        tmp=np.around(tmp/luminance_quant_tbl)

        # Z字形扫描
        tmp=tmp.reshape([64])
        tmp=np.array([tmp[zigzagOrder[k]] for k in range(64)]).astype(np.int32)

        if flag==0:
            print(tmp)
        flag+=1
        # DC系数差分编码
        DCs=tmp[0]-before
        before=tmp[0]

        # DC系数编码
        sosBitStream.write(huffmanEncode.encodeDCToBoolList(DCs,1),bool)

        # AC系数编码
        huffmanEncode.encodeACBlock(sosBitStream,tmp[1:],1)

#### 开始封装图像格式

In [ ]:
jpegFile = open(outputImgname, 'wb+')
# SOI字段，图像数据开始--20B
jpegFile.write(huffmanEncode.hexToBytes('FFD8FFE000104A46494600010100000100010000'))

In [ ]:
# DQT字段，写入相应量化表
# 写入亮度量化表--69B
jpegFile.write(huffmanEncode.hexToBytes('FFDB004300'))
luminance_quant_tbl = luminance_quant_tbl.reshape([64])
jpegFile.write(bytes(luminance_quant_tbl.tolist()))

# 写入色度量化表
# jpegFile.write(huffmanEncode.hexToBytes('FFDB004301'))
# chrominanceQuantTbl = chrominanceQuantTbl.reshape([64])
# jpegFile.write(bytes(chrominanceQuantTbl.tolist()))


In [11]:
# SOF0字段，图像帧开始--13B
jpegFile.write(huffmanEncode.hexToBytes('FFC0000B08'))
hHex = hex(srcHeight)[2:].rjust(4,'0')
wHex = hex(srcWidth)[2:].rjust(4,'0')
jpegFile.write(huffmanEncode.hexToBytes(hHex))  # 写入高度和宽度
jpegFile.write(huffmanEncode.hexToBytes(wHex))
jpegFile.write(huffmanEncode.hexToBytes('01011100'))

4

In [ ]:
# DHT字段，写入huffman表--420B
jpegFile.write(huffmanEncode.hexToBytes("FFC401A2"))
jpegFile.write(huffmanEncode.hexToBytes('0000000701010101010000000000000000040503020601000708090A0B0100020203010101010100000000000000010002030405060708090A0B1000020103030204020607030402060273010203110400052112314151061361227181143291A10715B14223C152D1E1331662F0247282F12543345392A2B26373C235442793A3B33617546474C3D2E2082683090A181984944546A4B456D355281AF2E3F3C4D4E4F465758595A5B5C5D5E5F566768696A6B6C6D6E6F637475767778797A7B7C7D7E7F738485868788898A8B8C8D8E8F82939495969798999A9B9C9D9E9F92A3A4A5A6A7A8A9AAABACADAEAFA110002020102030505040506040803036D0100021103042112314105511361220671819132A1B1F014C1D1E1234215526272F1332434438216925325A263B2C20773D235E2448317549308090A18192636451A2764745537F2A3B3C32829D3E3F38494A4B4C4D4E4F465758595A5B5C5D5E5F5465666768696A6B6C6D6E6F6475767778797A7B7C7D7E7F738485868788898A8B8C8D8E8F839495969798999A9B9C9D9E9F92A3A4A5A6A7A8A9AAABACADAEAFA'))

In [13]:
# 这个对jpeg的头解释的很详细了：https://blog.csdn.net/ymlbright/article/details/44179891
# SOS字段，扫描数据开始
jpegFile.write(huffmanEncode.hexToBytes('FFDA0008010100003F00'))

# 写入编码数据，若不是8的整数倍，先进行填充
sosLength = sosBitStream.__len__()
filledNum = 8 - sosLength % 8
if(filledNum!=0):
    sosBitStream.write(np.ones([filledNum]).tolist(),bool)
sosBytes = sosBitStream.read(bytes)
for i in range(len(sosBytes)):
    jpegFile.write(bytes([sosBytes[i]]))
    if(sosBytes[i]==255):
        jpegFile.write(bytes([0])) # FF to FF 00

# EOI字段，写入文件结束符
jpegFile.write(bytes([255,217])) # FF D9
jpegFile.close()